In [ ]:
%pip install langchain

In [ ]:
%pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith

In [ ]:
%pip install transformers

In [ ]:
%pip install torch

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st

openai_api_key = os.getenv("OPENAI_API_KEY")


def main():
    # streamlit으로 제목과 input box 생성
    st.title("논문 초록 작성기")
    content = st.text_input("논문의 초록을 작성할 주제와 간단한 내용을 입력해주세요.")

    # 언어모델 불러오기
    llm = ChatOpenAI(openai_api_key=openai_api_key)
    prompt = ChatPromptTemplate.from_messages(
        [("system", "You are a world class paper writer."), ("user", "{input}")]
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser

    # 버튼 클릭시 논문 초록 생성
    if st.button("논문 초록 작성하기"):
        with st.spinner("초록 작성 중입니다..."):
            result = chain.invoke(
                {"input": f"{content}에 대한 논문의 초록을 작성해줘."}
            )
            st.write(result)

In [ ]:
import httpx

In [ ]:
network = ""
collection_id = "0xed5af388653567af2f388e6224dc7c4b3241c544"

In [ ]:
collection_url = "https://api.reservoir.tools/collections/v7"
attribute_url = (
    f"https://api.reservoir.tools/collections/{collection_id}/attributes/all/v4"
)

headers = {
    "x-api-key": os.getenv("RESERVOIR_API_KEY"),
}

In [ ]:
params = {
    "id": collection_id,
}
collection = httpx.get(collection_url, headers=headers, params=params).json()

collection

In [ ]:
from langchain_community.document_loaders import ImageCaptionLoader


loader = ImageCaptionLoader(collection["collections"][0]["sampleImages"])
list_nfts = loader.load()
list_contents = [docs.page_content for docs in list_nfts]

In [ ]:
list_contents

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "NFT 이미지에 대한 설명을 제공해드릴려고 합니다. 이 설명들을 취합하여서 100자 내외로 요약해주세요.",
        ),
        ("user", "{list_contents}"),
    ]
)

llm = ChatOpenAI(
    temperature=0.8,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)
output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
resp = chain.invoke({"list_contents": list_contents})

In [ ]:
resp

In [ ]:
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 NFT 및 블록체인 전문가입니다. 특정 NFT 및 NFT 이미지에 대한 설명을 전달해드립니다. 설명을 정리하여 설명 구분없이 100자 이상으로 친근한 어투로 상세히 설명해주세요.",
        ),
        ("user", "이미지 설명: {image_desc}, NFT 설명: {nft_desc}"),
    ]
)

chain = prompt | llm

In [ ]:
chain.invoke(
    {"image_desc": resp, "nft_desc": collection["collections"][0]["description"]}
)

In [ ]:
nft_list_url = "https://api.reservoir.tools/tokens/v7"
params = {
    "collection": collection_id,
}

nft_list = httpx.get(nft_list_url, headers=headers, params=params).json()
nft_list

In [ ]:
images = [nft["token"]["imageSmall"] for nft in nft_list["tokens"]]

images

In [ ]:
loader = ImageCaptionLoader(images)
list_nfts = loader.load()
list_contents = [docs.page_content for docs in list_nfts]
list_contents